In [128]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
from itertools import combinations

## Import Data

In [129]:
cap_gen_df = pd.read_csv('../data/EIA/monthly_state_capacity_generation.csv')

## Clean Data

In [130]:
merged_df = cap_gen_df.copy()

In [133]:
# Change column names
merged_df.rename(columns = {'Wind Cap': 'wind', 'Est Tot Solar Cap': 'solar', 'All Sectors Gen': 'all_sec_gen',
                           'Biomass Cap': 'biomass', 'Geothermal Cap': 'geo', 'Nuclear Cap': 'nuclear',
                           'Conventional Hydroelectric Cap': 'hydro', 'Renewable Sources Cap': 'renew', 
                           'Fossil Fuels Cap': 'fossil'}, 
                 inplace = True)

In [134]:
# Remove NA rows in capacity
merged_df = merged_df[~merged_df.iloc[:,:-6].isnull().any(axis = 1)]

In [135]:
# Select only states (filter out regions)
states_list = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

merged_df = merged_df.query('State in @states_list').copy()

In [136]:
# Add month variable
merged_df['Month'] = merged_df['Date'].apply(lambda x: x.split(' ')[0])

## Regressions

### Pairs

In [137]:
def convert_coefficients(coeffs):
    ''' Input coeffs consists of [delta*v, (1-delta)*v, -(1/2)*v*p*delta]
    log(Y) = (v/p)*log(delta*K^(-p) + (1-delta)*K^(-p))
    ~= log(Y) = v*delta*log(K) + v*(1-delta)*log(L) - (1/2)*p*v*delta*(1-delta)(log(K) - log(L))^2 
    where delta is the share parameter for the first input
          v is the scale parameter 
          p is the elasticity parameter (elas = 1 / (1+p))
    '''
    
    # delta / (1-delta)
    temp_1 = coeffs[0]/coeffs[1]
    delta = temp_1 / (temp_1 + 1)
    
    # v 
    v = coeffs[0] / delta
    
    # -(1/2)*v*p*delta  / (v*delta)
    temp_2 = coeffs[2] / coeffs[0]
    p = temp_2 / (-1/2)
    sigma = 1 / (1 + p)
    
    return [delta, v, sigma]

In [138]:
fits = {}

In [139]:
fits['wind_solar'] = smf.ols('np.log(all_sec_gen+1) ~ np.log(solar+1) + np.log(wind+1) + np.power((np.log(solar+1) - np.log(wind+1)), 2)', 
        data = merged_df).fit()
fits['wind_solar'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     np.log(all_sec_gen + 1)   R-squared:                       0.112
Model:                                 OLS   Adj. R-squared:                  0.110
Method:                      Least Squares   F-statistic:                     49.42
Date:                     Thu, 03 Jan 2019   Prob (F-statistic):           4.47e-30
Time:                             12:44:26   Log-Likelihood:                -1674.0
No. Observations:                     1174   AIC:                             3356.
Df Residuals:                         1170   BIC:                             3376.
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
=======================================================================================================================
                                                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
Intercept                                               7.4933      0.088     85.178      0.000       7.321       7.666
np.log(solar + 1)                                       0.1752      0.015     11.972      0.000       0.146       0.204
np.log(wind + 1)                                        0.0033      0.010      0.346      0.729      -0.016       0.022
np.power((np.log(solar + 1) - np.log(wind + 1)), 2)     0.0127      0.002      5.975      0.000       0.009       0.017
==============================================================================
Omnibus:                      101.769   Durbin-Watson:                   0.138
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              128.547
Skew:                          -0.751   Prob(JB):                     1.22e-28
Kurtosis:                       3.611   Cond. No.                         66.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [140]:
fits['biomass_nuclear'] = smf.ols('np.log(all_sec_gen+1) ~ np.log(biomass+1) + np.log(nuclear+1) + np.power((np.log(biomass+1) - np.log(nuclear+1)), 2)', 
        data = merged_df).fit()
fits['biomass_nuclear'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     np.log(all_sec_gen + 1)   R-squared:                       0.411
Model:                                 OLS   Adj. R-squared:                  0.409
Method:                      Least Squares   F-statistic:                     272.0
Date:                     Thu, 03 Jan 2019   Prob (F-statistic):          6.72e-134
Time:                             12:44:27   Log-Likelihood:                -1433.5
No. Observations:                     1174   AIC:                             2875.
Df Residuals:                         1170   BIC:                             2895.
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
============================================================================================================================
                                                               coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------
Intercept                                                    7.5072      0.066    114.000      0.000       7.378       7.636
np.log(biomass + 1)                                          0.0462      0.017      2.660      0.008       0.012       0.080
np.log(nuclear + 1)                                          0.1557      0.009     17.555      0.000       0.138       0.173
np.power((np.log(biomass + 1) - np.log(nuclear + 1)), 2)    -0.0073      0.003     -2.626      0.009      -0.013      -0.002
==============================================================================
Omnibus:                       46.783   Durbin-Watson:                   0.120
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               52.038
Skew:                          -0.479   Prob(JB):                     5.01e-12
Kurtosis:                       3.384   Cond. No.                         39.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [141]:
fits['geo_hydro'] = smf.ols('np.log(all_sec_gen+1) ~ np.log(geo+1) + np.log(hydro+1) + np.power((np.log(geo+1) - np.log(hydro+1)), 2)', 
        data = merged_df).fit()
fits['geo_hydro'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     np.log(all_sec_gen + 1)   R-squared:                       0.082
Model:                                 OLS   Adj. R-squared:                  0.080
Method:                      Least Squares   F-statistic:                     34.88
Date:                     Thu, 03 Jan 2019   Prob (F-statistic):           1.37e-21
Time:                             12:44:27   Log-Likelihood:                -1693.8
No. Observations:                     1174   AIC:                             3396.
Df Residuals:                         1170   BIC:                             3416.
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
======================================================================================================================
                                                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
Intercept                                              7.4405      0.108     68.865      0.000       7.229       7.652
np.log(geo + 1)                                       -0.1385      0.042     -3.335      0.001      -0.220      -0.057
np.log(hydro + 1)                                      0.2317      0.043      5.371      0.000       0.147       0.316
np.power((np.log(geo + 1) - np.log(hydro + 1)), 2)    -0.0101      0.005     -2.203      0.028      -0.019      -0.001
==============================================================================
Omnibus:                      131.597   Durbin-Watson:                   0.127
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              192.341
Skew:                          -0.817   Prob(JB):                     1.71e-42
Kurtosis:                       4.124   Cond. No.                         159.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [143]:
fits['rnw_ff'] = smf.ols('np.log(all_sec_gen+1) ~ np.log(renew+1) + np.log(fossil+1) + np.power((np.log(renew+1) - np.log(fossil+1)), 2)', 
        data = merged_df).fit()
fits['rnw_ff'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     np.log(all_sec_gen + 1)   R-squared:                       0.925
Model:                                 OLS   Adj. R-squared:                  0.925
Method:                      Least Squares   F-statistic:                     4804.
Date:                     Thu, 03 Jan 2019   Prob (F-statistic):               0.00
Time:                             12:44:36   Log-Likelihood:                -224.21
No. Observations:                     1174   AIC:                             456.4
Df Residuals:                         1170   BIC:                             476.7
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
=========================================================================================================================
                                                            coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------
Intercept                                                -0.8410      0.082    -10.265      0.000      -1.002      -0.680
np.log(renew + 1)                                         0.4254      0.013     31.623      0.000       0.399       0.452
np.log(fossil + 1)                                        0.6283      0.011     59.108      0.000       0.607       0.649
np.power((np.log(renew + 1) - np.log(fossil + 1)), 2)     0.0645      0.004     17.552      0.000       0.057       0.072
==============================================================================
Omnibus:                       11.722   Durbin-Watson:                   0.461
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               14.374
Skew:                          -0.144   Prob(JB):                     0.000756
Kurtosis:                       3.459   Cond. No.                         124.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [144]:
results = []

for fit_type, fit_data in fits.items():
    
    results.append([fit_type] + convert_coefficients(list(fit_data.params[1:])))

In [145]:
pd.DataFrame(results, columns = ['pair', 'delta', 'scale', 'elasticity'])

,pair,delta,scale,elasticity
0,biomass_nuclear,0.228874,0.201970,0.759279
1,wind_solar,0.981347,0.178512,1.170320
2,geo_hydro,-1.486088,0.093211,1.170211
3,rnw_ff,0.403753,1.053686,1.434816


### All Renewables

In [74]:
## Construct terms for regression
renewables = ['solar', 'wind', 'geo', 'biomass', 'nuclear', 'hydro']

# Log of each source
beta_terms = ' + '.join(['np.log({0} + 1)'.format(x) for x in renewables])

# Log source_i * Log source_j
gamma_ij_terms = ' + '.join(['np.log({0} + 1)*np.log({1}+1)'.format(x, y) for x, y in combinations(renewables, 2)])

# ( Log source_i )^2
gamma_jj_terms = ' + '.join(['np.power(np.log({0} + 1), 2)'.format(x) for x in renewables])

In [75]:
fit_renew = smf.ols('np.log(all_sec_gen+1) ~ ' + ' + '.join([beta_terms, gamma_ij_terms, gamma_jj_terms]), 
        data = merged_df).fit()

fit_renew.summary()

In [67]:
# Collect results from fit
results = []

for source in renewables:
    
    gamma = fit_renew.params['np.power(np.log({0} + 1), 2)'.format(source)]
    beta = fit_renew.params['np.log({0} + 1)'.format(source)]
    results.append([source, gamma, beta])

In [77]:
# Convert results to elasticity estimates
results_df = pd.DataFrame(results, columns = ['source', 'gamma', 'beta'])

v = results_df['beta'].sum()
results_df['phi'] = results_df.apply(lambda x: 2*x.gamma / ((x.beta**2 / v) - x.beta), axis = 1)

results_df['elasticity'] = np.divide(1, np.add(1, results_df['phi']))

results_df

,source,gamma,beta,phi,elasticity
0,solar,-0.009156,0.044906,0.374732,0.727415
1,wind,0.037297,0.231355,-0.221655,1.284778
2,geo,0.096891,-1.215748,-0.114760,1.129638
3,biomass,-0.025211,0.949380,0.018534,0.981803
4,nuclear,0.042516,-0.518207,-8.983104,-0.125265
5,hydro,0.016418,-0.000597,55.086331,0.017830


#### Mixed Effects (State clusters)

In [104]:
fit_renew_mixed = smf.mixedlm('np.log(all_sec_gen+1) ~ ' + ' + '.join([beta_terms, gamma_ij_terms, gamma_jj_terms]), 
        data = merged_df, groups = merged_df["State"]).fit()
fit_renew_mixed.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                      Mixed Linear Model Regression Results
==================================================================================
Model:                 MixedLM     Dependent Variable:     np.log(all_sec_gen + 1)
No. Observations:      1174        Method:                 REML                   
No. Groups:            50          Scale:                  0.0231                 
Min. group size:       14          Likelihood:             314.5643               
Max. group size:       24          Converged:              Yes                    
Mean group size:       23.5                                                       
----------------------------------------------------------------------------------
                                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------------
Intercept                                6.621    1.211  5.466 0.000  4.247  8.995
np.log(solar + 1)                       -0.020    0.053 -0.376 0.707 -0.124  0.084
np.log(wind + 1)                         0.054    0.130  0.416 0.677 -0.200  0.308
np.log(geo + 1)                          0.521    0.745  0.700 0.484 -0.939  1.981
np.log(biomass + 1)                      0.026    0.358  0.073 0.942 -0.676  0.728
np.log(nuclear + 1)                     -0.155    0.161 -0.960 0.337 -0.471  0.161
np.log(hydro + 1)                        0.001    0.211  0.003 0.998 -0.412  0.414
np.log(solar + 1):np.log(wind + 1)      -0.005    0.003 -1.581 0.114 -0.011  0.001
np.log(solar + 1):np.log(geo + 1)        0.008    0.009  0.862 0.389 -0.010  0.026
np.log(solar + 1):np.log(biomass + 1)   -0.000    0.009 -0.027 0.979 -0.018  0.018
np.log(solar + 1):np.log(nuclear + 1)    0.003    0.004  0.798 0.425 -0.005  0.012
np.log(solar + 1):np.log(hydro + 1)     -0.008    0.006 -1.362 0.173 -0.021  0.004
np.log(wind + 1):np.log(geo + 1)        -0.231    0.095 -2.437 0.015 -0.418 -0.045
np.log(wind + 1):np.log(biomass + 1)    -0.015    0.025 -0.589 0.556 -0.063  0.034
np.log(wind + 1):np.log(nuclear + 1)    -0.010    0.011 -0.862 0.389 -0.031  0.012
np.log(wind + 1):np.log(hydro + 1)      -0.003    0.017 -0.181 0.856 -0.037  0.031
np.log(geo + 1):np.log(biomass + 1)      0.057    0.078  0.721 0.471 -0.097  0.210
np.log(geo + 1):np.log(nuclear + 1)     -0.001    0.069 -0.010 0.992 -0.137  0.135
np.log(geo + 1):np.log(hydro + 1)        0.126    0.078  1.618 0.106 -0.027  0.278
np.log(biomass + 1):np.log(nuclear + 1)  0.027    0.021  1.298 0.194 -0.014  0.068
np.log(biomass + 1):np.log(hydro + 1)    0.002    0.043  0.058 0.954 -0.081  0.086
np.log(nuclear + 1):np.log(hydro + 1)    0.004    0.021  0.172 0.864 -0.038  0.045
np.power(np.log(solar + 1), 2)           0.009    0.003  2.558 0.011  0.002  0.015
np.power(np.log(wind + 1), 2)            0.024    0.011  2.246 0.025  0.003  0.045
np.power(np.log(geo + 1), 2)            -0.013    0.090 -0.148 0.882 -0.189  0.162
np.power(np.log(biomass + 1), 2)        -0.010    0.027 -0.358 0.720 -0.063  0.043
np.power(np.log(nuclear + 1), 2)         0.033    0.015  2.115 0.034  0.002  0.063
np.power(np.log(hydro + 1), 2)           0.003    0.021  0.132 0.895 -0.039  0.045
Group Var                                0.606    0.988                           
==================================================================================

"""

In [105]:
# Collect results from fit
results = []

for source in renewables:
    
    gamma = fit_renew_mixed.params['np.power(np.log({0} + 1), 2)'.format(source)]
    beta = fit_renew_mixed.params['np.log({0} + 1)'.format(source)]
    results.append([source, gamma, beta])

In [106]:
# Convert results to elasticity estimates
results_df = pd.DataFrame(results, columns = ['source', 'gamma', 'beta'])

v = results_df['beta'].sum()
results_df['phi'] = results_df.apply(lambda x: 2*x.gamma / ((x.beta**2 / v) - x.beta), axis = 1)

results_df['elasticity'] = np.divide(1, np.add(1, results_df['phi']))

results_df

,source,gamma,beta,phi,elasticity
0,solar,0.008769,-0.019927,0.840865,0.543223
1,wind,0.023979,0.053932,-1.017774,-56.261123
2,geo,-0.013303,0.521074,-0.231578,1.301369
3,biomass,-0.009687,0.026116,0.790164,0.558608
4,nuclear,0.032625,-0.154834,0.309260,0.763790
5,hydro,0.002833,0.000577,-9.838597,-0.113140


#### Mixed Effects (Month Cluster)

In [122]:
fit_renew_mixed = smf.mixedlm('np.log(all_sec_gen+1) ~ ' + ' + '.join([beta_terms, gamma_ij_terms, gamma_jj_terms]), 
        data = merged_df, groups = merged_df["Month"]).fit()
fit_renew_mixed.summary()

/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", Conve

<class 'statsmodels.iolib.summary2.Summary'>
"""
                      Mixed Linear Model Regression Results
==================================================================================
Model:                 MixedLM     Dependent Variable:     np.log(all_sec_gen + 1)
No. Observations:      1174        Method:                 REML                   
No. Groups:            12          Scale:                  0.2962                 
Min. group size:       92          Likelihood:             -1059.6335             
Max. group size:       100         Converged:              No                     
Mean group size:       97.8                                                       
----------------------------------------------------------------------------------
                                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------------
Intercept                                4.821    0.373 12.920 0.000  4.090  5.552
np.log(solar + 1)                        0.192    0.054  3.588 0.000  0.087  0.297
np.log(wind + 1)                        -0.094    0.057 -1.663 0.096 -0.205  0.017
np.log(geo + 1)                         -3.207    0.381 -8.418 0.000 -3.953 -2.460
np.log(biomass + 1)                      0.741    0.119  6.217 0.000  0.508  0.975
np.log(nuclear + 1)                     -0.344    0.049 -7.097 0.000 -0.440 -0.249
np.log(hydro + 1)                        0.096    0.042  2.307 0.021  0.014  0.178
np.log(solar + 1):np.log(wind + 1)       0.016    0.004  3.642 0.000  0.007  0.024
np.log(solar + 1):np.log(geo + 1)       -0.023    0.017 -1.381 0.167 -0.055  0.010
np.log(solar + 1):np.log(biomass + 1)   -0.057    0.010 -5.841 0.000 -0.076 -0.038
np.log(solar + 1):np.log(nuclear + 1)    0.036    0.005  7.644 0.000  0.027  0.045
np.log(solar + 1):np.log(hydro + 1)     -0.009    0.005 -1.743 0.081 -0.019  0.001
np.log(wind + 1):np.log(geo + 1)         0.473    0.059  8.062 0.000  0.358  0.587
np.log(wind + 1):np.log(biomass + 1)    -0.061    0.010 -5.882 0.000 -0.081 -0.040
np.log(wind + 1):np.log(nuclear + 1)     0.004    0.003  1.137 0.256 -0.003  0.010
np.log(wind + 1):np.log(hydro + 1)      -0.008    0.005 -1.606 0.108 -0.019  0.002
np.log(geo + 1):np.log(biomass + 1)      0.001    0.024  0.023 0.981 -0.047  0.048
np.log(geo + 1):np.log(nuclear + 1)     -0.247    0.030 -8.363 0.000 -0.305 -0.189
np.log(geo + 1):np.log(hydro + 1)       -0.132    0.022 -6.068 0.000 -0.175 -0.089
np.log(biomass + 1):np.log(nuclear + 1)  0.033    0.006  5.225 0.000  0.020  0.045
np.log(biomass + 1):np.log(hydro + 1)   -0.026    0.012 -2.253 0.024 -0.049 -0.003
np.log(nuclear + 1):np.log(hydro + 1)    0.001    0.004  0.292 0.771 -0.008  0.010
np.power(np.log(solar + 1), 2)          -0.008    0.005 -1.570 0.116 -0.018  0.002
np.power(np.log(wind + 1), 2)            0.057    0.003 18.445 0.000  0.051  0.063
np.power(np.log(geo + 1), 2)             0.331    0.044  7.513 0.000  0.244  0.417
np.power(np.log(biomass + 1), 2)        -0.020    0.009 -2.337 0.019 -0.037 -0.003
np.power(np.log(nuclear + 1), 2)         0.021    0.005  4.441 0.000  0.012  0.031
np.power(np.log(hydro + 1), 2)           0.010    0.005  1.815 0.069 -0.001  0.020
Group Var                                0.005    0.006                           
==================================================================================

"""

In [125]:
# Collect results from fit
results = []

for source in renewables:
    
    gamma = fit_renew_mixed.params['np.power(np.log({0} + 1), 2)'.format(source)]
    beta = fit_renew_mixed.params['np.log({0} + 1)'.format(source)]
    results.append([source, gamma, beta])

In [126]:
# Convert results to elasticity estimates
results_df = pd.DataFrame(results, columns = ['source', 'gamma', 'beta'])

v = results_df['beta'].sum()
results_df['phi'] = results_df.apply(lambda x: 2*x.gamma / ((x.beta**2 / v) - x.beta), axis = 1)

results_df['elasticity'] = np.divide(1, np.add(1, results_df['phi']))

results_df

,source,gamma,beta,phi,elasticity
0,solar,-0.008192,0.192377,0.079331,0.926500
1,wind,0.056610,-0.094014,1.249189,0.444605
2,geo,0.330542,-3.206802,-0.911599,11.312042
3,biomass,-0.020318,0.741402,0.042704,0.959045
4,nuclear,0.021450,-0.344468,0.143432,0.874560
5,hydro,0.009662,0.096145,-0.193858,1.240476
